In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
import sqlite3

In [2]:
path = '/Applications/Driver/chromedriver_mac64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [3]:
url = 'https://www.booking.com/searchresults.en-gb.html?ss=New+York&ssne=New+York&ssne_untouched=New+York&label=gen173nr-1FCAEoggI46AdIM1gEaGyIAQGYAQm4AQfIAQ3YAQHoAQH4AQOIAgGoAgO4AoP1mbcGwAIB0gIkMTEyN2NhOWUtNzYwYS00MDVmLWEyODctZTEwYTQ2MmFjYjIx2AIG4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20088325&dest_type=city&checkin=2024-10-07&checkout=2024-10-10&group_adults=1&no_rooms=1&group_children=0'

In [4]:
wait = WebDriverWait(driver, 20)

driver.get(url)
for i in range(1,20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    wait.until(EC.element_to_be_clickable((By.XPATH,"//span[contains(text(), 'Load more results')]"))).click()


In [9]:
def is_element_present(htl, by, value):
    try:
        htl.find_element(by, value)
        return True
    except NoSuchElementException:
        return False

In [ ]:
# Lists to store hotel names and prices
hotel_names = []
hotel_prices = []
hotel_stars = []
hotel_rating = []

all_hotels = driver.find_elements(By.CSS_SELECTOR, "div[class='c624d7469d a0e60936ad a3214e5942 b0db0e8ada']")

print(len(set(all_hotels)))

for hotel in all_hotels:
    rating = "0"
    name = hotel.find_element(By.CSS_SELECTOR,"div[class='f6431b446c a15b38c233']")
    price = hotel.find_element(By.CSS_SELECTOR,"span[class='f6431b446c fbfd7c1165 e84eb96b1f']")
    stars = len(hotel.find_elements(By.CSS_SELECTOR,"span[class='fcd9eec8fb d31eda6efc c25361c37f']"))
    if is_element_present(hotel, By.CSS_SELECTOR,"div[class='a3b8729ab1 d86cee9b25']"):
        rating = hotel.find_element(By.CSS_SELECTOR,"div[class='a3b8729ab1 d86cee9b25']").find_element(By.CSS_SELECTOR,"div[class='ac4a7896c7']").text[7:]
    
    if name and price and stars:
        hotel_names.append(name.text)
        hotel_prices.append(price.text)
        hotel_stars.append(stars)
        hotel_rating.append(rating)

print(hotel_names)

# Create a DataFrame and save to CSV
booking = pd.DataFrame({
    'Hotel Name': hotel_names,
    'Price': hotel_prices,
    'Star': hotel_stars,
    'Rating': hotel_rating
})
print(booking)


## PUSHING DATA TO CSV

In [13]:
booking.to_csv('booking_nyc.csv', index=False)

## PUSHING DATA TO DATABASE

In [ ]:
# Establish a connection to SQLite database
engine = create_engine('sqlite:///hotel_booking_nyc.db', echo=True)

# Step 3: Insert DataFrame into the database table
booking.to_sql('booking_hotel_table', con=engine, if_exists='replace', index=False)

# Step 4: Verify the data insertion
conn = sqlite3.connect('hotel_booking_nyc.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM booking_hotel_table")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()